In [12]:
# Uncomment below if you want to run this file only
%run main.ipynb
#%run data_cleaning.ipynb
#%run data_visualization.ipynb
#%run feature_engineering.ipynb

In [13]:
# Read the CSV From FEATURE ENGINEERING data source file from S3 into a DataFrame
# Use the methods from the S3Utils class
if s3_utils.check_file_exists(output_file_key_data_feature_engineering):
    data = s3_utils.read_csv_from_s3(output_file_key_data_feature_engineering)

In [18]:
# Separate features and target
data=dataset
X = data.drop('target', axis=1)  # Replace 'target' with the name of your target column
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:

# Impute missing values in the training set
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

# Scale the imputed training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)

# Resample the training set using SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

# Train an SVM classifier
model = SVC(kernel='linear', class_weight='balanced')
model.fit(X_train_res, y_train_res)

# Impute missing values in the test set using the same imputer
X_test_imputed = imputer.transform(X_test)

# Scale the imputed test data using the same scaler
X_test_scaled = scaler.transform(X_test_imputed)

# Make predictions on the scaled test set
predictions = model.predict(X_test_scaled)




In [24]:
# Create a DataFrame with indices from X_test
predictions_df = pd.DataFrame(predictions, index=X_test.index, columns=['Predicted'])

# Save the DataFrame to a CSV file with the index
predictions_df.to_csv('svm_predictions.csv', index=True)

d:\Users\presc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [25]:

# Evaluate the model
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))


[[33888 31425]
 [ 1257  4468]]
              precision    recall  f1-score   support

           0       0.96      0.52      0.67     65313
           1       0.12      0.78      0.21      5725

    accuracy                           0.54     71038
   macro avg       0.54      0.65      0.44     71038
weighted avg       0.90      0.54      0.64     71038



In [26]:

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [1, 0.1, 0.01],
    'kernel': ['rbf', 'linear']
}

grid_search = GridSearchCV(svm.SVC(class_weight='balanced'), param_grid, refit=True, verbose=2)
grid_search.fit(X_train_res, y_train_res)

# Evaluate the best model found by grid search
best_model = grid_search.best_estimator_
best_predictions = best_model.predict(X_test)
print(classification_report(y_test, best_predictions))


Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END ........................C=0.1, gamma=1, kernel=rbf; total time=484.2min


In [1]:
best_parameters = grid_search.best_params_
print("Best parameters:", best_parameters)

NameError: name 'grid_search' is not defined